# Demo While Activity with ShiftAmount Activity
The combination of a while activity with a shift amount activity can be used to represent the loading or unloading of a vessel, where coordination of the process is represented on the granularity of the amount shifted in one iteration.

In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of Sites

In [2]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic

# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie",  # The name of the site
    "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5b",  # For logging purposes
    "geometry": location_from_site,  # The coordinates of the project site
    "capacity": 10,  # The capacity of the site
    "level": 2,
}  # The actual volume of the site

# Information on the dumping site - the "to site" - the "dump locatie"
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_to_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Dumplocatie",  # The name of the site
    "ID": "6dbbbdf5-4589-11e9-82b2-b469212bff5b",  # For logging purposes
    "geometry": location_to_site,  # The coordinates of the project site
    "capacity": 10,  # The capacity of the site
    "level": 0,
}  # The actual volume of the site (empty of course)

# The two objects used for the simulation
from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

init
level: 2
completed init
init
level: 0
completed init


## Creation of Vessel

In [3]:
# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.ContainerDependentMovable,  # A moving container, so capacity and location
        core.Processor,  # Allow for loading and unloading
        core.HasResource,  # Add information on serving equipment
        core.HasCosts,  # Add information on costs
        core.LoadingFunction,  # Add a loading function
        core.UnloadingFunction,  # Add an unloading function
    ),
    {},
)

# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: 10

# TSHD variables
data_hopper = {
    "env": my_env,  # The simpy environment
    "name": "Hopper 01",  # Name
    "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",  # For logging purposes
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 5,  # Capacity of the hopper - "Beunvolume"
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}

hopper = TransportProcessingResource(**data_hopper)

init
level: 0
completed init


## Definition of Shift Amount activity

In [4]:
shift_amount_activity_loading_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer MP",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff52",  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": from_site,
    "destination": hopper,
    "amount": 1,
    "duration": 20,
    "postpone_start": True,
}
activity = model.ShiftAmountActivity(**shift_amount_activity_loading_data)

while_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "while",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5g",  # For logging purposes
    "registry": registry,
    "sub_process": activity,
    "condition_event": [{"type":"container", "concept": hopper, "state":"full"}],
    "postpone_start": False,
}
while_activity = model.WhileActivity(**while_data)


get_full_event : default
start get_available
start event instance None


## Run simulation

In [5]:
my_env.run()


conditional 
start event instance None
start : {}
{'origin.6dbbbdf4-4589-11e9-a501-b469212bff5b': 2, 'destination.6dbbbdf6-4589-11e9-95a2-b469212bff5b': 5}
put_callback - id_ default
{}
put_callback - id_ default
{}
put_callback - id_ default
{'default': {5: <Event() object at 0x27a3f069a88>}}
amount :5
destination request : {<simpy.resources.resource.Resource object at 0x0000027A3F067A08>: <Request() object at 0x27a3ea83088>}
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x0000027A3F067A08>: <Request() object at 0x27a3ea83088>}
site request : {<simpy.resources.resource.Resource object at 0x0000027A3F067A08>: <Request() object at 0x27a3ea83088>, <simpy.resources.resource.Resource object at 0x0000027A3F067088>: <Request() object at 0x27a3ea832c8>}
end requestIfAvailable : {<simpy.resources.resource.Resource object at 0x0000027A3F067A08>: <Request() object at 0x27a3ea83088>, <simpy.resources.resource.Resource object at 0x0000027A3F067088>: <Request

RuntimeError: Attempting to shift content from an empty origin or to a full destination. ({'origin.6dbbbdf4-4589-11e9-a501-b469212bff5b': 0, 'destination.6dbbbdf6-4589-11e9-95a2-b469212bff5b': 3})

In [6]:
log_df = pd.DataFrame(while_activity.log)
data =log_df[['Message', 'ActivityState', 'Timestamp', 'Value', 'ActivityID']]
data

,Message,ActivityState,Timestamp,Value,ActivityID
0,conditional process while,START,1970-01-01 01:00:00,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
1,sub process Transfer MP,START,1970-01-01 01:00:00,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
2,Transfer MP,START,1970-01-01 01:00:00,1,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
3,Transfer MP,STOP,1970-01-01 01:00:20,1,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
4,sub process Transfer MP,STOP,1970-01-01 01:00:20,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
5,sub process Transfer MP,START,1970-01-01 01:00:20,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
6,Transfer MP,START,1970-01-01 01:00:20,1,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
7,Transfer MP,STOP,1970-01-01 01:00:40,1,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
8,sub process Transfer MP,STOP,1970-01-01 01:00:40,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5g
9,sub process Transfer MP,START,1970-01-01 01:00:40,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5g


The resulting levels of objects in the hopper and the from_site are requested below. 

In [7]:
hopper.container.get_level()

2

In [8]:
from_site.container.get_level()

0